# Data Loading

In [ ]:
!pip install 'pycaret[full]'

In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error,accuracy_score, r2_score
from pycaret.regression import*
from google.colab import files

drive.mount('/content/drive')

dfVanco = pd.read_excel('/content/drive/MyDrive/data/original_dataset.xlsx')

In [ ]:
!pip install shap

In [ ]:
import warnings
import numba

warnings.filterwarnings("ignore", category=numba.NumbaDeprecationWarning)

import pandas as pd
import numpy as np
import shap
import math
from pycaret.regression import*
from math import isnan
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
print(dfVanco)

# Pre-processing

In [ ]:
# This code is rounding certain columns to a specified decimal place using the round() function in pandas

dfVanco['BMI'] = round(dfVanco['BMI'], 1)
dfVanco['WBC'] = round(dfVanco['WBC'], 1)
dfVanco['Crcl'] = round(dfVanco['Crcl'], 1)
dfVanco['Clvanco'] = round(dfVanco['Clvanco'], 1)
dfVanco['Half_life'] = round(dfVanco['Half_life'], 1)
dfVanco['Ke'] = round(dfVanco['Ke'], 3)

In [ ]:
dfVanco[:3]

In [ ]:
dfVanco=dfVanco.drop(columns='Case_no')
dfVanco[:3]

In [ ]:
dfVanco['Initial VCM_daily_dose'].plot.hist()
dfVanco['Initial VCM_daily_dose'].describe()

# Outlier Removal

In [ ]:
# To remove outliers, remove the top and bottom 5% of the dataset

q1 = dfVanco['Initial VCM_daily_dose'].quantile(0.05)
q2 = dfVanco['Initial VCM_daily_dose'].quantile(0.5)
q3 = dfVanco['Initial VCM_daily_dose'].quantile(0.95)
iqr = q3 - q1

print(f'iqr: {iqr}')
print(f'low threshold : {q1-1.5*iqr}')
print(f'high theshold : {q3+1.5*iqr}')

In [ ]:
outlier1 = dfVanco['Initial VCM_daily_dose']>(q3+1.5*iqr)
outlier2 = dfVanco['Initial VCM_daily_dose']<(q1-1.5*iqr)

In [ ]:
a = list(dfVanco[outlier2].index)
b = list(dfVanco[outlier1].index)
a.extend(b)
df1 = dfVanco.drop(a)

# df1 is the dataset after removing the top and bottom 5% of the dataset to remove outliers
print(f'# data: {len(df1)}')
df1['Initial VCM_daily_dose'].plot.hist(bins=10)

In [ ]:
df1.describe()

# Modeling training using pycaret, an auto machine library

In [ ]:
reg1 = setup(data=df1, target='Initial VCM_daily_dose',
             train_size = 0.8,
             fold = 5, fold_shuffle = True,
             use_gpu=True,
             session_id = 42
            )

# Evaluation

In [ ]:
best = compare_models(sort='mse')

In [ ]:
best_tune = tune_model(best)

In [ ]:
evaluate_model(best_tune)

In [ ]:
predictions = predict_model(best_tune)
predictions[:5]

In [ ]:
df_result = pd.DataFrame()
df_result['true'] = predictions['Initial VCM_daily_dose']
df_result['prediction'] = predictions['prediction_label']

# Since vancomycin is prescribed in units of 100 anyway, the numerical value is rounded up to 100.
def round_to_nearest_fifty(number):
    return round(number / 50.0) * 50

rounded_values = [round_to_nearest_fifty(num) for num in predictions['prediction_label']]
predictions["prediction_label"]= rounded_values
predictions

In [ ]:
df_result['prediction']

In [ ]:
df_result['prediction']=predictions["prediction_label"]
df_result['prediction']

# Results

In [ ]:
df_result

In [ ]:
df_result['accuracy'] = np.where(abs(df_result['true'] - df_result['prediction']) <=50, 1, 0)
accuracy = round((len(df_result.loc[df_result['accuracy'] == 1])/ len(df_result)) * 100, 1)
print('Accuracy: ', accuracy, '%')


In [ ]:
plot_model(best_tune, plot = 'error')

# Log Transformation of 'Initial VCM_daily_dose'

In [ ]:
# Applying a log transformation to the 'Initial VCM_daily_dose' column in the df_D2 dataframe.
# This process is often used in data science to manage skewed data or to linearize relationships that are exponential in nature.

In [ ]:
df1['Initial VCM_daily_dose_log'] = df1['Initial VCM_daily_dose'].apply(lambda x: math.log(x))

In [ ]:
df2=df1.drop(columns='Initial VCM_daily_dose')

In [ ]:
df2

In [ ]:
reg1 = setup(data=df2, target='Initial VCM_daily_dose_log',
             train_size = 0.8,
             fold = 5, fold_shuffle = True,
             use_gpu=True,
             session_id = 42
            )

In [ ]:
best = compare_models(sort='mse')

In [ ]:
best_tune = tune_model(best)

In [ ]:
evaluate_model(best_tune)

In [ ]:
predictions = predict_model(best_tune)

In [ ]:
predictions

In [ ]:
df_result = pd.DataFrame()
df_result['Initial VCM_daily_dose_log'] = predictions['Initial VCM_daily_dose_log']
df_result['prediction_label_log']=predictions['prediction_label']
df_result

In [ ]:
index_list = [135, 115, 131, 55, 95, 29, 157, 51, 101, 145, 19, 85, 15, 66, 24, 30, 132, 105, 152, 16, 75, 18, 12, 9, 31, 155, 98, 56, 134, 160, 139, 78, 60]
index_series = pd.Series(index_list)  # Convert the list to a pandas Series

# List comprehension to store values
value_list = [df1.loc[index, 'Initial VCM_daily_dose'] for index in index_series]

print(value_list)


In [ ]:
df_result['Initial VCM_daily_dose']=value_list
df_result

In [ ]:
df_result['prediction'] = np.exp(df_result['prediction_label_log'])

In [ ]:
df_result

In [ ]:
# round off the prediction values to the nearest fifty
rounded_values = [round_to_nearest_fifty(num) for num in df_result['prediction']]
df_result['prediction']= rounded_values


In [ ]:
df_result

In [ ]:
df_result['accuracy'] = np.where(abs(df_result['Initial VCM_daily_dose'] - df_result['prediction']) <=50, 1, 0)
accuracy = round((len(df_result.loc[df_result['accuracy'] == 1])/ len(df_result)) * 100, 1)
print('Accuracy: ', accuracy, '%')

# Dataset augmentation

In [ ]:
df1 = df1.drop(columns=['Initial VCM_daily_dose_log'])

In [ ]:
# Dataset imbalance check

percentage = (df1['Initial VCM_daily_dose'].value_counts(normalize=True).loc[2000] * 100)
print(f"The percentage of data with 'Initial VCM_daily_dose' = 2000 is {percentage.round(1)} %")

In [ ]:
# Check the dataset except for the value of ['Initial VCM_daily_dose']  = 2000
df_temp = df1[df1['Initial VCM_daily_dose'] != 2000]


# the number of remaining data after erasing
print(f'# data: {len(df_temp)}')

# check with histogram
df_temp['Initial VCM_daily_dose'].plot.hist(bins=10)

In [ ]:
# Amplify other datasets to match data imbalance
# n times data amplification: Amplify the original data using the data 'without' the initial dose value of 2000
# After many attempts, it is reasonable to amplify it by about 4 times.

n = 4
df_list = [df_temp] * n
df_iter = pd.concat(df_list)
df_D = pd.concat([df1, df_iter])


# the number of data after amplification
print(f'# data: {len(df_D)}')

# check with histogram
df_D['Initial VCM_daily_dose'].plot.hist(bins=10)

In [ ]:
df_D

In [ ]:
augmented_dataset = pd.ExcelWriter('augmented_dataset.xlsx')

In [ ]:
df_D.to_excel(augmented_dataset, index=False)

In [ ]:
augmented_dataset.save()

In [ ]:
files.download('augmented_dataset.xlsx')


# Model training with amplified data

In [ ]:
# The amplified data is randomly corrupted and stored
df_D = df_D.sample(frac=1).reset_index(drop=True)

reg2 = setup(data=df_D,
             target='Initial VCM_daily_dose',
             train_size=0.8,
             fold=5,
             fold_shuffle=True,
             use_gpu=True,
             session_id=42
            )


# Evaluation

In [ ]:
# find and tune the best model
best = compare_models(sort='mse')

In [ ]:
best_tune = tune_model(best)

In [ ]:
# evaluate and make predictions with the best model
evaluate_model(best_tune)

In [ ]:
# assuming the data is already preprocessed and the setup is complete
ensemble = ensemble_model(best_tune, method = 'Bagging')

In [ ]:
# evaluate and make predictions with the bagging model
evaluate_model(ensemble)

In [ ]:
predictions = predict_model(best_tune)
predictions2 = predict_model(ensemble)

In [ ]:
predictions

In [ ]:
# round off the prediction values to the nearest fifty
rounded_values = [round_to_nearest_fifty(num) for num in predictions['prediction_label']]

predictions["prediction_label"]= rounded_values

# create a dataframe to hold the true and predicted values
df_result = pd.DataFrame()
df_result['true'] = predictions['Initial VCM_daily_dose']
df_result['prediction'] = predictions['prediction_label']

In [ ]:
predictions

In [ ]:
df_result['prediction']

# Results

In [ ]:
df_result['accuracy'] = np.where(abs(df_result['true'] - df_result['prediction']) <= 50, 1, 0)
accuracy = round((len(df_result.loc[df_result['accuracy'] == 1])/ len(df_result)) * 100, 1)
print('Accuracy: ', accuracy, '%')


In [ ]:
plot_model(best_tune, plot = 'error')

# Feature engineering using SHAP

In [ ]:
# create a shap explainer object
explainer = shap.Explainer(best_tune)

In [ ]:
# calculate shap values on your data
shap_values = explainer.shap_values(df_D)

In [ ]:
# plot the shap values
shap.summary_plot(shap_values, df_D, plot_type="bar" )

In [ ]:
# get feature importance
feature_importance = np.abs(shap_values).mean(axis=0)
feature_importance = pd.Series(feature_importance, index=df_D.columns)

# get top 10 important features
top_10_features = feature_importance.sort_values(ascending=False)[:10].index

# create a new dataframe with only the top 10 important features
df_D_important = df_D[top_10_features]

In [ ]:
# get indices of the selected features
selected_features_indices = [df_D.columns.tolist().index(feature) for feature in top_10_features]

# plot SHAP values of the selected features
shap.summary_plot(shap_values[:, selected_features_indices], df_D[top_10_features], plot_type="bar" )

# Feature selection based on pycaret & SHAP feature importance

In [ ]:
Pycaret_Feature_importance = ['Weight', 'Vd', 'Age', 'BUN', 'Half_life', 'eGFR', 'Height','Clvanco', 'Ke', 'ACEi' ]
SHAP_Feature_importance = ['Clvanco', 'CRP', 'eGFR', 'ARB', 'Age', 'PLT','SCr', 'Weight', 'Hb','Gender']

# Convert the lists to sets and union them
union_set = set(Pycaret_Feature_importance) | set(SHAP_Feature_importance)

# Convert the result back to a list
union_list = list(union_set)
union_list

In [ ]:
df_D2 = df_D[union_list].copy()
df_D2 = df_D2[['Gender', 'Age', 'Weight', 'Height', 'Hb', 'PLT', 'CRP', 'eGFR', 'BUN', 'SCr','ACEi', 'ARB', 'Clvanco', 'Vd', 'Ke', 'Half_life']]
df_D2['Initial VCM_daily_dose'] = df_D['Initial VCM_daily_dose']
df_D2

In [ ]:
reg3 = setup(data=df_D2,
             target='Initial VCM_daily_dose',
             train_size=0.8,
             fold=5,
             fold_shuffle=True,
             use_gpu=True,
             session_id=42
            )


In [ ]:
best = compare_models(sort='mse')

In [ ]:
best_tune = tune_model(best)

In [ ]:
# evaluate and make predictions with the best model
evaluate_model(best_tune)

In [ ]:
predictions = predict_model(best_tune)

In [ ]:
# round off the prediction values to the nearest fifty
rounded_values = [round_to_nearest_fifty(num) for num in df_result['prediction']]

predictions["prediction_label"]= rounded_values

# create a dataframe to hold the true and predicted values
df_result = pd.DataFrame()
df_result['true'] = predictions['Initial VCM_daily_dose']
df_result['prediction'] = predictions['prediction_label']

In [ ]:
predictions

In [ ]:
df_result['accuracy'] = np.where(abs(df_result['true'] - df_result['prediction']) <= 50, 1, 0)
accuracy = round((len(df_result.loc[df_result['accuracy'] == 1])/ len(df_result)) * 100, 1)
print('Accuracy: ', accuracy, '%')


In [ ]:
plot_model(best_tune, plot = 'error')

In [ ]:
explainer = shap.Explainer(best_tune)
shap_values = explainer.shap_values(df_D2)
shap.summary_plot(shap_values, df_D2, plot_type="bar" )

# Log Transformation of augmented dataset of 'Initial VCM_daily_dose'

In [ ]:
df_D2['Initial VCM_daily_dose']

In [ ]:
df_D2['Initial VCM_daily_dose_log'] = df_D2['Initial VCM_daily_dose'].apply(lambda x: math.log(x))
df_D2

In [ ]:
df_D2['Initial VCM_daily_dose_log'].plot.hist(bins=10)

In [ ]:
df_D3=df_D2.drop(columns='Initial VCM_daily_dose')

In [ ]:
reg4 = setup(data=df_D3,
             target='Initial VCM_daily_dose_log',
             train_size=0.8,
             fold=5,
             fold_shuffle=True,
             use_gpu=True,
             session_id=42
            )


In [ ]:
best = compare_models(sort='mse')

In [ ]:
best_tune = tune_model(best)

In [ ]:
# evaluate and make predictions with the best model
evaluate_model(best_tune)

In [ ]:
predictions = predict_model(best_tune)

In [ ]:
predictions

In [ ]:
df_result = pd.DataFrame()
df_result['Initial VCM_daily_dose_log'] = predictions['Initial VCM_daily_dose_log']
df_result['prediction_label_log']=predictions['prediction_label']

In [ ]:
index_list = [145, 280, 175, 373, 420, 73, 132, 137, 30, 72, 70, 94, 316, 90, 376, 416, 9, 247, 196, 231, 192, 239, 350, 228, 55, 356, 56, 298, 272, 79, 331, 116, 208, 194, 340, 184, 218, 39, 168, 364, 75, 76, 419, 33, 113, 402, 148, 283, 371, 15, 78, 261, 0, 19, 271, 336, 281, 278, 77, 82, 360, 104, 25, 383, 172, 370, 312, 42, 173, 274, 124, 296, 297, 374, 22, 46, 285, 93, 410, 412, 57, 415, 24, 17, 66]
index_series = pd.Series(index_list)  # Convert the list to a pandas Series

# List comprehension to store values
value_list = [df_D2.loc[index, 'Initial VCM_daily_dose'] for index in index_series]

print(value_list)



In [ ]:
df_result['Initial VCM_daily_dose']=value_list
df_result

In [ ]:
df_result['prediction'] = np.exp(df_result['prediction_label_log'])

In [ ]:
df_result

In [ ]:
# round off the prediction values to the nearest fifty
rounded_values = [round_to_nearest_fifty(num) for num in df_result['prediction']]
df_result['prediction']= rounded_values
df_result

In [ ]:
df_result['accuracy'] = np.where(abs(df_result['Initial VCM_daily_dose'] - df_result['prediction']) < 100, 1, 0)
accuracy = round((len(df_result.loc[df_result['accuracy'] == 1])/ len(df_result)) * 100, 1)
print('Accuracy: ', accuracy, '%')

In [ ]:
plot_model(best_tune, plot = 'error')

In [ ]:
final_VancoAI = finalize_model(estimator=best_tune)

In [ ]:
save_model(model=final_VancoAI,
           model_name='final_VancoAI',
           verbose=False)